In [1]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 32 kB/s 


In [2]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from catboost.datasets import rotten_tomatoes
from sklearn.model_selection import train_test_split as train
from pandas import read_csv as read

In [3]:
path = "carsWithAddInfo.csv"
data = read(path, delimiter=",")

In [4]:
data.head(5)

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,colorH,colorS,colorV,imgSharpness,imgResX,imgResY
0,-1,volvo 264gl,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3062,ohc,four,141,mpfi,3.78,3.15,9.5,114,5400,19,25,22625.0,30.0,47.0,32.0,318.0,960.0,1280.0
1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,353.0,73.0,62.0,965.0,1080.0,1920.0,NaN
2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,358.0,88.0,57.0,992.0,1050.0,1680.0,NaN
3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,224.0,40.0,37.0,910.0,780.0,1170.0,NaN
4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,177.0,32.0,57.0,709.0,684.0,1024.0,NaN


In [5]:
cat_text = ['CarName','fueltype','aspiration','doornumber',
'carbody','enginelocation','drivewheel','enginetype','cylindernumber','fuelsystem']

def hasher(inp): 
  newInp = 0
  for i in range(0,len(inp)):
    newInp += ord(inp[i])*(i+1)
  return newInp


for z in range(0,len(cat_text)):
  for j in range(0,len(data)):
    data.loc[j,cat_text[z]] = hasher(data.loc[j,cat_text[z]])

In [6]:
data.head(5)

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,colorH,colorS,colorV,imgSharpness,imgResX,imgResY
0,-1,5351,642,1639,1131,1555,652,1683,109.1,188.8,68.9,55.5,3062,616,1131,141,1059,3.78,3.15,9.5,114,5400,19,25,22625.0,30.0,47.0,32.0,318.0,960.0,1280.0
1,3,17042,642,647,687,7037,652,1683,88.6,168.8,64.1,48.8,2548,1030,1131,130,1059,3.47,2.68,9.0,111,5000,21,27,353.0,73.0,62.0,965.0,1080.0,1920.0,NaN
2,3,19586,642,647,687,7037,652,1683,88.6,168.8,64.1,48.8,2548,1030,1131,130,1059,3.47,2.68,9.0,111,5000,21,27,358.0,88.0,57.0,992.0,1050.0,1680.0,NaN
3,1,30569,642,647,687,4584,652,1683,94.5,171.2,65.5,52.4,2823,1088,685,152,1059,2.68,3.47,9.0,154,5000,19,26,224.0,40.0,37.0,910.0,780.0,1170.0,NaN
4,2,4858,642,647,1131,1555,640,1683,99.8,176.6,66.2,54.3,2337,616,1131,109,1059,3.19,3.40,10.0,102,5500,24,30,177.0,32.0,57.0,709.0,684.0,1024.0,NaN


In [17]:
#data2 = data.drop(data.columns[14:18], axis=1)
data2=data


data3 = data.drop(data.columns[24:25], axis=1)

X = data3.values[::, 0:27]
y = data2.values[::, 24:25]


X_train, X_test, y_train, y_test = train(X, y, test_size=0.6)

clf = CatBoostRegressor(iterations=1400,learning_rate=0.03,od_type='Iter',od_wait=1500,
                             depth=3,random_strength=1,
                            l2_leaf_reg=10,
                             sampling_frequency='PerTree',
                             
                            )
res = clf.fit(X_train, y_train,
             eval_set=(X_test,y_test),
             use_best_model=True,
             verbose=False)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

-123.62411522500634
-173.6105095680162


In [18]:
sr = 0
ind = 0
for a, b in zip(clf.predict(X_test), y_test):
    #print(a, b)
    #print(abs((b-a)/b * 100))
    if b != 0:
      sr+=abs((b-a)/max(b,a) * 100)
      ind+=1
sr/=ind
sr=100-sr
print(sr)

[51.417153551935925]


In [8]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(clf, f)

In [54]:
clf.save_model("model")

In [11]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

-162.8271175668611
-151.97277486335096
